# Advanced SQL

## Lecture Objectives

- Connect to a database from within a python program and run queries
- Understand psycopg2's cursors and commits
- Generate dynamic queries

## Relational Database Management System (RDBMS)

#### It is a persistent data storage system
 - Survives after the process in which it was created has ended
 - Is written to non-volatile storage
 
#### RDBMS is the de facto standard for storing data
 - Examples: Oracle, MySQL, SQLServer, Postgres
 - Non-relational models are used for some applications (including data science)


## Why RDBMS?

#### An RDBMS provides the ability to
 - **Model** relations in data
 - **Query** data and their relations efficiently
 - **Maintain** data consistency and integrity

It will require a Data Model


#### RDBMS Data Model

 - **Schema** defines the structure of the data
 - The **database** is composed of a number of user-defined **tables**
 - Each **table** will have **columns** (aka fields) and **rows** (aka records)
 - A column is of a given **data type**
 - A row is an entry in a table with data for each column of that table


#### RDBMS and SQL

 - SQL is the language used to query relational databases
 - **All RDBMS** use SQL and the syntax and keywords are the same for the most part, across systems
 - **SQL is used to interact** with RDBMS, allowing you to create tables, alter tables, insert records, update records, delete records, and query records within and across tables.
 - Even non-relational databases like **Hadoop** usually have a SQL-like interface available.

#### Database Life Cycle (DBLC) 

#### An inefficient way to store data...

##### A single table with records of customer purchases at an outdoor sports store.

#### Relational Database Management Systems

 - A RDBMS is a type of database where **data is stored in multiple related tables.**
 - The tables are related through **primary** and **foreign keys**.
 - The same information as shown before in an RDBMS:


#### Primary Keys

 - Every table in a RDBMS has a **primary key**  that uniquely identifies that row
 - Each entry must have a primary key, and primary keys cannot repeat within a table
 - Primary keys are usually integers but can take other forms


#### Foreign Keys and Table Relationships

 - A **foreign key** is a column that uniquely identifies a column in another table
 - Often, a foreign key in one table is a primary key in another table
 - We can use foreign keys to join tables


#### Entity Relationship Diagram (ERD)

#### Database Normalization

##### Store each piece of information in exactly one place
 - Details about a user (address, age) are only stored once (in a users table)
 - Any other table (eg. purchases) where this data might be relevant, only references the user_id

##### Why is this a good thing?


#### Normal Forms

Normalization is explained through *Normal forms*

 - 1st Normal Form: each attribute has a single value
 - 2nd Normal Form: no dependencies on part of a key
 - 3rd Normal Form: no transitive dependencies


#### Why Not Normalize?

Sometimes databases are not fully normalized.

 - Data structure not known/may change
 - Writing a schema/converting data is hard
 - Simple queries are important
 - Data will not be changed/integrity not important
 - Storage is cheap


#### Data Science in the DBLC

 - DS / Operations: querying, aggregating
 - DS / Implementing: identifying, cleaning, pushing external data sources inside a RDBMS
 - DS / Design: recommendations on the model, specs on operations

## Combining SQL and Python

Often you will find yourself working with data that are only accessable through SQL.  However, your machine-learning capabilities are built in Python.  To resolve this issue, we can simply set up a connection from Python to the SQL database to bring the data to us.

## Why do we care?

- SQL-based databases are extremely common in almost all industry environments
- Can leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
- Very useful for scaled data pipelines, pre-cleaning, data exploration
- Allows for dynamic query generation and hence automations

## psycopg2

- A Python library that allows for connections with PostgresSQL databases to easily query and retrieve data for analysis.
- [Documentation--Includes Installation Instructions](http://initd.org/psycopg/docs/install.html)
- In addition to what's listed in the documentation, if you have the anaconda distribution of Python 
```python 
conda install psycopg2 
```
should worked
- There are similar packages for other flavors of SQL that work much the same way

## General Workflow

1. Establish connection to Postgres database using psycopg2
2. Create a cursor
3. Use the cursor to execute SQL queries and retrieve data
4. Commit SQL actions
4. Close the cursor and connection

# Walkthrough 1: Creating a database from Python

### Connect to the database
- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you need to create a database, you can first connect to Postgres using the dbname 'postgres' to initialize

In [489]:
import psycopg2
conn = psycopg2.connect(dbname='postgres', host='localhost')

### Instantiate the Cursor

- A cursor is a control structure that enables traversal over the records in a database
- Executes and fetches data
- When the cursor points at the resulting output of a query, it can only read each observation once.  If you choose to see a previously read observation, you must rerun the query. 
- Can be closed without closing the connection

In [490]:
cur = conn.cursor()

### Commits

- Data changes are not actually stored until you choose to commit
- You can choose to have automatic commit by using ` autocommit = True`
- When connecting directly to the Postgres Server to initiate server level commands such as creating a database, you must use the `autocommit = True` option since Postgres does not have "temporary" transactions at the database level

In [491]:
conn.autocommit = True

###  Create a database

In [492]:
cur.execute('DROP DATABASE IF EXISTS temp;')
cur.execute('CREATE DATABASE temp;')

### Disconnect from the cursor and database
- Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

In [493]:
cur.close() # This is optional
conn.close() # Closing the connection also closes all cursors

# Walkthrough 2: Using the new database

### Connect to our database

In [412]:
conn = psycopg2.connect(dbname='temp', host='localhost')

In [373]:
cur = conn.cursor()

### Create a new table

In [374]:
query = '''
        CREATE TABLE logins (
            userid integer, 
            tmstmp timestamp, 
            type varchar(10)
        );
        '''
cur.execute(query)

### Insert data into new table

`os` is needed to get the current directory and, later, dynamically identify the files we need to insert using listdir.

In [375]:
import os

In [376]:
path = os.getcwd()

In [377]:
path

'/Users/mattzeidenberg/lectures/sql-python'

In [378]:
query = '''
        COPY logins 
        FROM '{0}/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        '''.format(path)
print(query)
cur.execute(query)


        COPY logins 
        FROM '/Users/mattzeidenberg/lectures/sql-python/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        


### Run a query to get 30 records from our data

In [379]:
query = '''
        SELECT *
        FROM logins
        LIMIT 30;
        '''
cur.execute(query)

### Lets look at our data one line at a time

In [380]:
cur.fetchone()

(579, datetime.datetime(2013, 11, 20, 3, 20, 6), 'mobile')

### Many lines at a time

In [381]:
#fetchmany(n) to get n rows
cur.fetchmany(10)

[(823, datetime.datetime(2013, 11, 20, 3, 20, 49), 'web'),
 (953, datetime.datetime(2013, 11, 20, 3, 28, 49), 'web'),
 (612, datetime.datetime(2013, 11, 20, 3, 36, 55), 'web'),
 (269, datetime.datetime(2013, 11, 20, 3, 43, 13), 'web'),
 (799, datetime.datetime(2013, 11, 20, 3, 56, 55), 'web'),
 (890, datetime.datetime(2013, 11, 20, 4, 2, 33), 'mobile'),
 (330, datetime.datetime(2013, 11, 20, 4, 54, 59), 'mobile'),
 (628, datetime.datetime(2013, 11, 20, 4, 57, 22), 'mobile'),
 (398, datetime.datetime(2013, 11, 20, 5, 3, 19), 'mobile'),
 (482, datetime.datetime(2013, 11, 20, 5, 4, 43), 'mobile')]

### Or everything at once

In [382]:
#fetchall() grabs all remaining rows
results = cur.fetchall()

In [383]:
type(results)

list

In [387]:
type(results[0])
print(len(results))
print(results)

[(581, datetime.datetime(2013, 11, 20, 5, 12, 3), 'mobile'), (370, datetime.datetime(2013, 11, 20, 5, 26, 46), 'mobile'), (230, datetime.datetime(2013, 11, 20, 5, 28, 29), 'web'), (596, datetime.datetime(2013, 11, 20, 5, 28, 36), 'web'), (274, datetime.datetime(2013, 11, 20, 5, 43, 8), 'mobile'), (581, datetime.datetime(2013, 11, 20, 5, 47, 10), 'web'), (417, datetime.datetime(2013, 11, 20, 5, 54, 37), 'mobile'), (185, datetime.datetime(2013, 11, 20, 5, 56, 22), 'mobile'), (371, datetime.datetime(2013, 11, 20, 5, 58, 35), 'mobile'), (133, datetime.datetime(2013, 11, 20, 5, 59, 7), 'web'), (621, datetime.datetime(2013, 11, 20, 6, 1, 46), 'web'), (306, datetime.datetime(2013, 11, 20, 6, 3, 23), 'mobile'), (509, datetime.datetime(2013, 11, 20, 6, 4, 43), 'web'), (505, datetime.datetime(2013, 11, 20, 6, 9, 52), 'web'), (678, datetime.datetime(2013, 11, 20, 6, 34, 18), 'web'), (889, datetime.datetime(2013, 11, 20, 6, 36, 32), 'mobile'), (202, datetime.datetime(2013, 11, 20, 6, 43, 33), 'mob

In [390]:
#cur.scroll(1)

### You can even iterate over the cursor

In [391]:
cur.execute(query)
for record in cur:
    print("{}: user {} logged in via {}".format(record[1], record[0], record[2]))

2013-11-20 03:20:06: user 579 logged in via mobile
2013-11-20 03:20:49: user 823 logged in via web
2013-11-20 03:28:49: user 953 logged in via web
2013-11-20 03:36:55: user 612 logged in via web
2013-11-20 03:43:13: user 269 logged in via web
2013-11-20 03:56:55: user 799 logged in via web
2013-11-20 04:02:33: user 890 logged in via mobile
2013-11-20 04:54:59: user 330 logged in via mobile
2013-11-20 04:57:22: user 628 logged in via mobile
2013-11-20 05:03:19: user 398 logged in via mobile
2013-11-20 05:04:43: user 482 logged in via mobile
2013-11-20 05:12:03: user 581 logged in via mobile
2013-11-20 05:26:46: user 370 logged in via mobile
2013-11-20 05:28:29: user 230 logged in via web
2013-11-20 05:28:36: user 596 logged in via web
2013-11-20 05:43:08: user 274 logged in via mobile
2013-11-20 05:47:10: user 581 logged in via web
2013-11-20 05:54:37: user 417 logged in via mobile
2013-11-20 05:56:22: user 185 logged in via mobile
2013-11-20 05:58:35: user 371 logged in via mobile
2013

# Dynamic Queries

- A Dynamic Query is a query that generates based on context.


### Example

We have 8 login csv files that we need to insert into the logins table.  Instead of doing a COPY FROM query 8 times, we should utilize Python (or any future languages) to make this more efficient.  This is possible due to tokenized strings.

### First lets get an idea of how many records we start with

In [392]:
cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

In [393]:
record_count

10000

In [394]:
type(record_count)

int

In [395]:
conn.commit()

### Create a query template and determine file path for imports

Use string formatting to generate a query for each approved file.

**[WARNING: BEWARE OF SQL INJECTION](http://initd.org/psycopg/docs/usage.html)**

NEVER use + or % or .format to reformat strings to be used with .execute

In [396]:
num = 579
terribly_unsafe = "SELECT * FROM logins WHERE userid = " + str(num) + ";"
print(terribly_unsafe)


date_cut = "2014-08-01"
horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s;" % date_cut
print(horribly_risky)
## Python is happy, but if num or date_cut included something malicious
## your data could be at risk

SELECT * FROM logins WHERE userid = 579;
SELECT * FROM logins WHERE tmstmp > 2014-08-01;


### What is an SQL Injection Attack?

In [397]:
date_cut = "2014-08-01; DROP TABLE logins" # The user enters a date in a field on a web form
horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s;" % date_cut
print(horribly_risky)

SELECT * FROM logins WHERE tmstmp > 2014-08-01; DROP TABLE logins;


### Practice safe SQL with Psycopg2

In [398]:
query = '''
        COPY logins 
        FROM %(file_path)s
        DELIMITER ','
        CSV;
        '''

In [399]:
path = os.getcwd()
folder_path = path + '/logins_data/'
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name != 'logins01.csv':
        file_path=folder_path+file_name
        cur.execute(query, {'file_path':file_path})
        print('{0} inserted into table.'.format(file_name))

logins08.csv inserted into table.
logins06.csv inserted into table.
logins07.csv inserted into table.
logins05.csv inserted into table.
logins04.csv inserted into table.
logins03.csv inserted into table.
logins02.csv inserted into table.


### Visit [bobby-tables.com](http://www.bobby-tables.com/) to learn more about SQL safety.



### Let's check the total number of records we have right now.

In [400]:
print("Old record count: {}".format(record_count))

cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("New record count: {}".format(record_count))

Old record count: 10000
New record count: 78588


### Transactions can be rolled back until they're committed

In [401]:
conn.rollback()

cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("After rollback: {}".format(record_count))

After rollback: 10000


### Don't forget to commit your changes

In [402]:
conn.commit()

### Close your connection

In [403]:
conn.close()

### Using With Statements

In [404]:
query = "SELECT count(*) FROM logins;"
with psycopg2.connect(dbname='temp', host='localhost') as conn:
    with conn.cursor() as curs:
        print("Cursor inside with block: {}".format(curs))
        curs.execute(query)
        record_count = curs.fetchone()[0]
        print(record_count)
    print("Cursor outside with block: {}".format(curs))
    

Cursor inside with block: <cursor object at 0x104870650; closed: 0>
10000
Cursor outside with block: <cursor object at 0x104870650; closed: -1>


### Note that the connection is *not* closed automatically:

In [405]:
conn

<connection object at 0x10490cb50; dsn: 'dbname=temp host=localhost', closed: 0>

In [406]:
conn.close()
conn

<connection object at 0x10490cb50; dsn: 'dbname=temp host=localhost', closed: 1>

# Key Things to Remember

* Connections must be established using an existing database, username, database IP/URL, and maybe passwords
* If you have no created databases, you can connect to Postgres using the dbname 'postgres' to initialize db commands
* Data changes are not actually stored until you choose to commit. This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions is only temporary stored.
* Autocommit = True is necessary to do database commands like CREATE DATABASE.  This is because Postgres does not have temporary transactions at the database level.
* If you ever need to build similar pipelines for other forms of database, there are libraries such PyODBC which operate very similarly.
* SQL connection databases utilizes cursors for data traversal and retrieval.  This is kind of like an iterator in Python.
* Cursor operations typically goes like the following:
    - execute a query
    - fetch rows from query result if it is a SELECT query
    - because it is iterative, previously fetched rows can only be fetched again by rerunning the query
    - close cursor through .close()
* Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

# Exercise

You're given a file called `playgolf.csv` in the data folder.  The file is comma delimited and the first row is the header.  Without opening and looking at the file, create a table and insert the data. Here is the header and first row:


|Date|Outlook|Temperature|Humidity|Windy|Result|
|----|-------|-----------|--------|-----|------|
|07-01-2014|sunny|85|85|false|Don't Play|

In [494]:
conn = psycopg2.connect(dbname='temp', host='localhost')

In [495]:
cur = conn.cursor()

In [496]:
query = '''
        CREATE TABLE playgolf (
            golfdate date,
            outlook varchar(20),
            temperature integer,
            humidity integer,
            windy varchar(20),
            result varchar(20)
        );
        '''
cur.execute(query)

In [497]:
import os
path = os.getcwd()
path

'/Users/mattzeidenberg/lectures/sql-python'

In [498]:
query = '''
        COPY playgolf
        FROM '{0}/playgolf.csv' 
        DELIMITER ',' 
        CSV HEADER;
        '''.format(path)
print(query)
cur.execute(query)


        COPY playgolf
        FROM '/Users/mattzeidenberg/lectures/sql-python/playgolf.csv' 
        DELIMITER ',' 
        CSV HEADER;
        


In [499]:
query = '''
        SELECT *
        FROM playgolf
        LIMIT 30;
        '''
cur.execute(query)

In [500]:
cur.fetchone()

(datetime.date(2014, 7, 1), 'sunny', 85, 85, 'FALSE', "Don't Play")

In [501]:
conn.commit()

In [502]:
cur.close()
conn.close()